In [1]:
import pandas as pd 
import os
import json
import numpy as np 

from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plte
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from category_encoders.ordinal import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE


from scipy.stats import pearsonr, spearmanr


from hashlib import sha256

In [2]:
df = pd.read_csv('./fraud_oracle.csv')

In [3]:
df

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15415,Nov,4,Friday,Toyota,Urban,Tuesday,Nov,5,Male,Married,...,6 years,31 to 35,No,No,External,none,no change,1 vehicle,1996,Collision
15416,Nov,5,Thursday,Pontiac,Urban,Friday,Dec,1,Male,Married,...,6 years,31 to 35,No,No,External,more than 5,no change,3 to 4,1996,Liability
15417,Nov,5,Thursday,Toyota,Rural,Friday,Dec,1,Male,Single,...,5 years,26 to 30,No,No,External,1 to 2,no change,1 vehicle,1996,Collision
15418,Dec,1,Monday,Toyota,Urban,Thursday,Dec,2,Female,Married,...,2 years,31 to 35,No,No,External,more than 5,no change,1 vehicle,1996,All Perils


In [4]:

age_groups = {
    '16-17': 'Young',
    '18-20': 'Young',
    '21-25': 'Young',
    '26-30': 'Young',
    '31-35': 'Middle-aged',
    '36-40': 'Middle-aged',
    '41-50': 'Middle-aged',
    '51-65': 'Senior',
    '65+': 'Senior'
}
df['AgeGroup'] = df['AgeOfPolicyHolder'].map(age_groups)
df.drop(columns=['AgeOfPolicyHolder'], inplace=True)


def categorize_range(value):
    if value == 'less than 20000':
        return 'low'
    elif value in ['20000 to 29000', '30000 to 39000']:
        return 'mid'
    elif value in ['40000 to 59000', '60000 to 69000', 'more than 69000']:
        return 'high'
    else:
        return 'mid'


df['VehiclePrice_Cat'] = df['VehiclePrice'].apply(categorize_range)


In [5]:

label_encoder = LabelEncoder()


col_ordering = [
    {'col':'PastNumberOfClaims','mapping':{'none':0 ,'1':1,'2 to 4':3,'more than 4':5 }},
    {'col':'NumberOfSuppliments','mapping':{'none':0,'1 to 2':1,'3 to 5':3,'more than 5':6}},
    {'col':'VehiclePrice','mapping':{'more than 69000':5,'20000 to 29000':1,'30000 to 39000':2,'less than 20000':0,
                                     '40000 to 59000':3,'60000 to 69000':4}},
    {'col':'AgeOfVehicle','mapping':{'new': 0, '2 years': 1, '3 years': 2, '4 years': 3, '5 years': 4, '6 years': 5, '7 years': 6, 'more than 7': 7}},
    {'col':'Year','mapping': {1994: 0, 1995: 1, 1996: 2}},
    {'col':'Days_Policy_Accident','mapping': {'none': 0, '1 to 7': 1,'8 to 15': 2,'15 to 30': 3, 'more than 30': 4}},
    {'col':'Days_Policy_Claim','mapping': {'none': 0, '1 to 7': 1,'8 to 15': 2,'15 to 30': 3, 'more than 30':4 }},
    {'col':'AddressChange_Claim','mapping': {'1 year': 1, 'no change': 0, '4 to 8 years': 4, '2 to 3 years': 2, 'under 6 months': 0.5}},
    {'col':'AgeGroup','mapping': {'Young': 0, 'Middle-aged': 1, 'Senior': 2}},
    {'col':'VehiclePrice_Cat','mapping': {'high': 2, 'mid': 1, 'low': 0}},
    {'col':'NumberOfCars','mapping': {'3 to 4': 3, '1 vehicle': 1, '2 vehicles': 2, '5 to 8': 7, 'more than 8': 9}},
]
ord_encoder = OrdinalEncoder(mapping = col_ordering, return_df=True)

df = ord_encoder.fit_transform(df)




for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = label_encoder.fit_transform(df[col])



c:\Users\Астемир\.pyenv\pyenv-win\versions\3.10.6\lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\Астемир\.pyenv\pyenv-win\versions\3.10.6\lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\Астемир\.pyenv\pyenv-win\versions\3.10.6\lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downc

In [6]:
df.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,AgeGroup,VehiclePrice_Cat
0,2,5,6,6,1,6,5,1,0,2,...,0,0,0,0,1,3,0,2,-1.0,2
1,4,3,6,6,1,2,5,4,1,2,...,1,0,0,0,0,1,0,1,-1.0,2
2,10,5,0,6,1,5,10,2,1,1,...,0,0,0,0,0,1,0,1,-1.0,2
3,6,2,2,17,0,1,6,1,1,1,...,1,0,0,6,0,1,0,2,-1.0,1
4,4,5,1,6,1,6,4,2,0,2,...,0,0,0,0,0,1,0,1,-1.0,2


In [7]:
df.rename(columns = {'FraudFound_P':'Fraud'},inplace = True)


In [8]:
df.shape

(15420, 34)

In [15]:

def add_gaussian_noise(df, columns, noise_level=0.01):
    for col in columns:
        noise = np.random.normal(0, noise_level, size=df[col].shape)
        df[col] += noise
    return df

def split_random_(df):

    x_1, x_2 = train_test_split(df[df.Fraud == 1], test_size=0.4, random_state=3)
    x_3, x_4 = train_test_split(df[df.Fraud == 0 ], test_size = 0.6, random_state=33)
    data_1 = pd.concat([x_1,x_3])
    data_2 = pd.concat([x_2, x_4])
    return data_1, data_2

def prepare_data(df, X_test):
    scale = MinMaxScaler()
    smote = SMOTE(random_state=22)

 
    X_train, y_train = df.drop(columns="Fraud"), df.Fraud
    X_train_scale = scale.fit_transform(X_train)
    X_test_scale = scale.transform(X_test)


    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_train_poly = poly.fit_transform(X_train_scale)


    X_smote, y_smote = smote.fit_resample(X_train_poly, y_train)

   
    X_test_poly = poly.transform(X_test_scale)

    return X_smote, y_smote, X_test_poly

def fit_predict(X_train, X_test, y_train, y_test):
    model = LogisticRegression(max_iter=250)
    roc_auc_cv = cross_val_score(model, X_train, y_train, cv=5, scoring="roc_auc")
    print(f"Кросс-валидация ROC-AUC: {roc_auc_cv}")
    print(f"Среднее значение ROC-AUC: {np.mean(roc_auc_cv):.4f}")

    model.fit(X_train, y_train)


    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)


    roc_auc_test = roc_auc_score(y_test, y_pred_proba)
    logloss_test = log_loss(y_test, y_pred_proba)
    accuracy_test = accuracy_score(y_test, y_pred)
    f1_test = f1_score(y_test, y_pred)

    params = model.coef_

    return params, logloss_test, roc_auc_test, accuracy_test, f1_test

---

* Глава 1 
    - Non - iid

---

In [9]:
X_, X_test, y_, y_test = train_test_split(df.drop(columns="Fraud"), df.Fraud, test_size=0.2, random_state=231)

In [11]:
df_clients = pd.concat([X_, y_],axis=1)
df_clients.to_csv("./datas/IID.csv",index=False)

pd.concat([X_test, y_test],axis=1).to_csv("./datas/TEST_SAMPLE.csv",index=False)



In [12]:
X_1,X_2, y_1,y_2 = train_test_split(X_, y_,train_size=0.5, random_state=231)

pd.concat([X_1, y_1], axis=1).to_csv("./datas/IID_1.csv",index=False)
pd.concat([X_2, y_2], axis=1).to_csv("./datas/IID_2.csv",index=False)

In [13]:
df_1 = pd.concat([X_1, y_1], axis=1)
df_2 = pd.concat([X_2, y_2], axis=1)

In [16]:
df_1, df_2 = split_random_(df_clients)



In [17]:
(df_1.shape, df_2.shape)

((5080, 34), (7256, 34))

In [19]:
col_1 = df_1.drop(columns="Fraud").columns
col_2 = df_2.drop(columns="Fraud").columns

In [25]:
noise_df_2 = add_gaussian_noise(df_2,
                                col_1,
                                noise_level=1.4)
noise_df_1 = add_gaussian_noise(df_1,
                                col_2,
                                noise_level=1.5)

In [27]:
noise_df_1.to_csv("./datas/NON_IID_FL_1.csv",index=False)

In [28]:
noise_df_2.to_csv("./datas/NON_IID_FL_2.csv",index=False)

In [22]:
noise_df_1.columns

Index(['Month', 'WeekOfMonth', 'DayOfWeek', 'Make', 'AccidentArea',
       'DayOfWeekClaimed', 'MonthClaimed', 'WeekOfMonthClaimed', 'Sex',
       'MaritalStatus', 'Age', 'Fault', 'PolicyType', 'VehicleCategory',
       'VehiclePrice', 'PolicyNumber', 'RepNumber', 'Deductible',
       'DriverRating', 'Days_Policy_Accident', 'Days_Policy_Claim',
       'PastNumberOfClaims', 'AgeOfVehicle', 'PoliceReportFiled',
       'WitnessPresent', 'AgentType', 'NumberOfSuppliments',
       'AddressChange_Claim', 'NumberOfCars', 'Year', 'BasePolicy', 'AgeGroup',
       'VehiclePrice_Cat', 'Fraud'],
      dtype='object')

In [17]:
X_train_1, y_train_1, X_test_1 = prepare_data(df_1, X_test)


In [19]:
_, loss_1, roc_auc_1, accuracy_1, f1_score_1 = fit_predict(X_train_1, X_test_1, y_train_1, y_test)
(loss_1, roc_auc_1, accuracy_1, f1_score_1)

(0.7717916020834025,
 0.6964398916551406,
 0.44163424124513617,
 0.15255905511811024)

In [20]:
X_train_2, y_train_2, X_test_2 = prepare_data(df_2, X_test)


In [21]:
_, loss_2, roc_auc_2, accuracy_2, f1_score_2 = fit_predict(X_train_2, X_test_2, y_train_2, y_test)
(loss_2, roc_auc_2, accuracy_2, f1_score_2)

(0.7322310144564195,
 0.671855189603504,
 0.5119974059662775,
 0.15591699383062255)

In [23]:
X_train_1_noise, y_train_1_noise, X_test_1_nosie = prepare_data(noise_df_1, X_test)
params, loss_1_noise, roc_auc_1_noise, accuracy_1_noise, f1_score_1_noise = fit_predict(X_train_1_noise, X_test_1_nosie, y_train_1_noise, y_test)
(loss_1_noise, roc_auc_1_noise, accuracy_1_noise, f1_score_1_noise)


Кросс-валидация ROC-AUC: [0.7857471  0.81034947 0.7908577  0.81475968 0.79857615]
Среднее значение ROC-AUC: 0.8001


(0.7757571354900864,
 0.6722509469508454,
 0.4066147859922179,
 0.14883720930232558)

In [19]:
params.size

594

In [48]:
X_train_2_noise, y_train_2_noise, X_test_2_nosie = prepare_data(noise_df_2, X_test)
params, loss_2_noise, roc_auc_2_noise, accuracy_2_noise, f1_score_2_noise = fit_predict(X_train_2_noise, X_test_2_nosie, y_train_2_noise, y_test)
(loss_2_noise, roc_auc_2_noise, accuracy_2_noise, f1_score_2_noise)

Кросс-валидация ROC-AUC: [0.85925062 0.87490975 0.86824104 0.87733464 0.8846021 ]
Среднее значение ROC-AUC: 0.8729


(0.6629745025902152,
 0.6688992929313038,
 0.5817120622568094,
 0.16558861578266496)

In [21]:
params.size

594

In [39]:
all_data_noise = pd.concat([noise_df_1, noise_df_2])

In [40]:
X_train_all_noise, y_train_all_noise, X_test_all_nosie = prepare_data(all_data_noise, X_test)
_, loss_all_noise, roc_auc_all_noise, accuracy_all_noise, f1_score_all_noise = fit_predict(X_train_all_noise, X_test_all_nosie, y_train_all_noise, y_test)
(loss_all_noise, roc_auc_all_noise, accuracy_all_noise, f1_score_all_noise)

(0.684003755692351,
 0.6916677374344645,
 0.5664721141374838,
 0.17316017316017315)

* Глава 2 
    - IID

In [170]:
X_1, X_2, y_1, y_2 = train_test_split(df_clients.drop(columns="Fraud"), df_clients.Fraud, test_size=0.5,random_state=2211)

In [174]:
df_1_iid = pd.concat([X_1, y_1],axis=1)

In [175]:
df_2_iid = pd.concat([X_2, y_2],axis=1) 

In [176]:
X_train_1_iid, y_train_1_iid, X_test_1_iid = prepare_data(df_1_iid, X_test)
loss_1, roc_auc_1, accuracy_1, f1_score_1 = fit_predict(X_train_1_iid, X_test_1_iid, y_train_1_iid, y_test)
(loss_1, roc_auc_1, accuracy_1, f1_score_1)

(0.4677620247839864,
 0.7986831489165516,
 0.7519455252918288,
 0.24778761061946902)

In [178]:
X_train_2_iid, y_train_2_iid, X_test_2_iid = prepare_data(df_2_iid, X_test)
loss_2, roc_auc_2, accuracy_2, f1_score_2 = fit_predict(X_train_2_iid, X_test_2_iid, y_train_2_iid, y_test)
(loss_2, roc_auc_2, accuracy_2, f1_score_2)

(0.4614940804301869,
 0.7858171968649148,
 0.7538910505836576,
 0.24477611940298508)

In [230]:
noise_df_1.to_csv('NON_IID_1.csv', index=False)
noise_df_2.to_csv('NON_IID_2.csv', index=False)

In [2]:
choices=["work_burnout_stress",
        "job_change_finding_stress",
        "accident_stress",
        "life_meaning",
        "death_fear"]

In [8]:
choices
start = "death_fear"

In [9]:
if start in choices:
    print(True)

else:
    print(False)

True
